In [1]:
import os
import random
import shutil
import time
import warnings
from datetime import datetime
import argparse

import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim

from torch.cuda.amp import autocast, GradScaler

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('logs')

SEED=1
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

GPU = 'cuda:0'
START_EPOCH = 0
ARCH = 'densenet'
EPOCHS = 10
LR = .0005
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
BATCH_SIZE = 100
WORKERS=2
#TRAINDIR="data/training/DEWP_class"
#VALDIR="data/test/DEWP_class"
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [2]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def train(train_loader, model, criterion, optimizer, scaler, epoch):

    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top2 = AverageMeter('Acc@2', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top2],
        prefix="Epoch: [{}]".format(epoch))

    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
                
        images = images.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        optimizer.zero_grad()
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        acc1, acc2 = accuracy(output, target, topk=(1, 2))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top2.update(acc2[0], images.size(0))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % PRINT_FREQ == 0:
            progress.display(i)

            writer.add_scalar('Train Acc@1',
                acc1[0],
                epoch * len(train_loader) + i)

def validate(val_loader, model, criterion, epoch):

    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top2 = AverageMeter('Acc@2', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top2],
        prefix='Test: ')

    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            images = images.cuda(non_blocking=True)
            target = target.cuda(non_blocking=True)
            
            output = model(images)
            loss = criterion(output, target)

            acc1, acc2 = accuracy(output, target, topk=(1, 2))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top2.update(acc2[0], images.size(0))
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

                writer.add_scalar('Validation Acc@1',
                    acc1[0],
                    epoch * len(val_loader) + i)

def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)

In [3]:
##From PyTorch docs https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html 

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [4]:
def single_init(train_dir, val_dir, num_classes):
    model_ft, IMG_SIZE = initialize_model(ARCH, num_classes, False, use_pretrained=False)

    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB)
    ])

    transform_val = transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB)
    ])

    torch.cuda.set_device(GPU)
    model_ft.cuda(GPU)
    criterion = nn.CrossEntropyLoss().cuda(GPU)
    scaler = GradScaler()

    optimizer = optim.SGD(
        model_ft.parameters(),
        lr=LR,
        weight_decay=WEIGHT_DECAY,
        momentum=MOMENTUM
    )
    
    model = model_ft

    # use CosineAnnealingLR
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=EPOCHS)
    
    train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)

    val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transform_val)
    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True) 

    return (model, criterion, optimizer, scheduler, scaler, train_loader, val_loader)

In [5]:
TEMP_CHECKPOINT_PATH = "./checkpoint_temp.pth.tar"
DEWP_CHECKPOINT_PATH = "./checkpoint_dewp.pth.tar"
VISIB_CHECKPOINT_PATH = "./checkpoint_visib.pth.tar"
WDSP_CHECKPOINT_PATH = "./checkpoint_wdsp.pth.tar"

TEMP_training_dir = "data/training/TEMP_class"
TEMP_test_dir = "data/test/TEMP_class"
DEWP_training_dir = "data/training/DEWP_class"
DEWP_test_dir = "data/test/DEWP_class"
VISIB_training_dir = "data/training/VISIB_class"
VISIB_test_dir = "data/test/VISIB_class"
WDSP_training_dir = "data/training/WDSP_class"
WDSP_test_dir = "data/test/WDSP_class"

In [7]:
model, criterion, optimizer, scheduler, scaler, train_loader, val_loader = single_init(DEWP_training_dir, DEWP_test_dir, num_classes=3)
    
start = datetime.now()

for epoch in range(START_EPOCH, EPOCHS):

    train(train_loader, model, criterion, optimizer, scaler, epoch)
    validate(val_loader, model, criterion, epoch)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, filename=DEWP_CHECKPOINT_PATH)

    scheduler.step()
    print('Learning rate: ' + str(scheduler.get_last_lr()))
    print("Time elapsed: " + str(datetime.now() - start))

Epoch: [0][ 0/17]	Time  1.057 ( 1.057)	Data  0.000 ( 0.000)	Loss 1.1461e+00 (1.1461e+00)	Acc@1  26.00 ( 26.00)	Acc@2  59.00 ( 59.00)
Test: [0/4]	Time  0.786 ( 0.786)	Loss 1.0625e+00 (1.0625e+00)	Acc@1  41.00 ( 41.00)	Acc@2  79.00 ( 79.00)
Learning rate: [0.0004877641290737884]
Time elapsed: 0:00:17.728993
Epoch: [1][ 0/17]	Time  0.858 ( 0.858)	Data  0.000 ( 0.000)	Loss 1.1020e+00 (1.1020e+00)	Acc@1  37.00 ( 37.00)	Acc@2  74.00 ( 74.00)
Test: [0/4]	Time  0.756 ( 0.756)	Loss 1.0457e+00 (1.0457e+00)	Acc@1  42.00 ( 42.00)	Acc@2  83.00 ( 83.00)
Learning rate: [0.0004522542485937368]
Time elapsed: 0:00:35.061094
Epoch: [2][ 0/17]	Time  0.867 ( 0.867)	Data  0.000 ( 0.000)	Loss 1.0705e+00 (1.0705e+00)	Acc@1  40.00 ( 40.00)	Acc@2  79.00 ( 79.00)
Test: [0/4]	Time  0.776 ( 0.776)	Loss 1.0700e+00 (1.0700e+00)	Acc@1  39.00 ( 39.00)	Acc@2  78.00 ( 78.00)
Learning rate: [0.0003969463130731183]
Time elapsed: 0:00:52.502455
Epoch: [3][ 0/17]	Time  0.872 ( 0.872)	Data  0.000 ( 0.000)	Loss 1.0442e+00 (1.